https://www.youtube.com/watch?v=tcqEUSNCn8I
https://github.com/pixegami/langchain-rag-tutorial/blob/main/query_data.py

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import os
import shutil
from langchain_community.llms import Ollama
import json
from langchain_core.prompts import ChatPromptTemplate

In [2]:
#paper_id = "2402.01383v1"
#paper_id = "2408.02304"
#paper_id = "2402.06196"
#paper_id = "2409.09957"
paper_id = "2409.15180"
CHROMA_PATH = paper_id+"/chroma"
#CHROMA_PATH = paper_id+"/chroma300"
DATA_PATH = paper_id
file = 'rag_largechroma'
#file = 'rag300'

In [3]:

def store_summary(survey1,filename):
    with open(f'summaries/'+paper_id+'/'+filename+'.txt', 'w') as file:
        file.write(survey1)
def write_json(documents_data,file_name):
    converted_data = [dict(item) for item in documents_data['source_documents']]
    dict_data = {'answer':documents_data['result'],'source':converted_data}
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)


### Create Chroma database

In [4]:

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

In [5]:
documents = load_documents()

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
documents[4]

Document(metadata={'source': '2409.15180/2311.15308v2.AV_Deepfake1M__A_Large_Scale_LLM_Driven_Audio_Visual_Deepfake_Dataset.pdf'}, page_content='4 2 0 2\n\nl u J\n\n9 2\n\n]\n\nV C . s c [\n\n2 v 8 0 3 5 1 . 1 1 3 2 : v i X r a\n\nAV-Deepfake1M: A Large-Scale LLM-Driven Audio-Visual Deepfake Dataset\n\nZhixi Cai Monash University Melbourne, Australia zhixi.cai@monash.edu\n\nShreya Ghosh Curtin University Perth, Australia shreya.ghosh@curtin.edu.au\n\nAman Pankaj Adatia Indian Institute of Technology Ropar Ropar, India 2020csb1154@iitrpr.ac.in\n\nMunawar Hayat Qualcomm San Diego, United States hayat@qti.qualcomm.com\n\nAbhinav Dhall Flinders University Adelaide, Australia abhinav.dhall@flinders.edu.au\n\nTom Gedeon Curtin University Perth, Australia tom.gedeon@curtin.edu.au\n\nKalin Stefanov Monash University Melbourne, Australia kalin.stefanov@monash.edu\n\nFake\n\n......\n\nthe terrible songbookand not unique\n\ngoing to\n\n.....................ReplacementDeletionInsertion...\n\nDeepe

In [7]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,         #300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [8]:
chunks = split_text(documents)

Split 21 documents into 2569 chunks.
The Codex model relies on Generative Pre-trained Transformer (GPT) models the company previously invented for text generation. The public code available on GitHub was used during the fine-tuning of the model to implement the code recognition and generation capabilities.

3 copilot.github.com 4 openai.com/blog/openai-codex 5 plugins.jetbrains.com/plugin/17718-github-copilot 6 github.com/features/codespaces

3

4

Burak Yetiştiren et al.

Features IDE Support

First Release Time Developer Providing References Suggestions Explanation of Suggestions Providing Multiple Sugges- tions

Training Data Source

to

ChatGPT No IDE Support

Nov-30-2022 OpenAI NO

YES NO (Theoretically user can manually ask for another suggestion.) GitHub Repositories, OpenAI Codex Dataset, other code repositories such as GitLab, Bitbucket, and SourceForge N/A

Amazon CodeWhisperer JetBrains, Visual Studio Code, AWS Cloud9, or the AWS Lambda console

June-23-2022 AWS YES

NO YES 

In [9]:
#
some = chunks[0:10]
#print(some)
print(len(chunks[11].page_content))
chunks[11].page_content

973


'June-23-2022 AWS YES\n\nNO YES (Up to 5)\n\n“Vast amounts of publicly available code"\n\nGitHub Copilot IntelliJ IDEA, Android Stu- dio, AppCode, CLion, Code With Me Guest, DataGrip, DataSpell, GoLand, Jet- Brains Client, MPS, Ph- pStorm, PyCharm, Rider, RubyMine, WebStorm Oct-29-2021 OpenAI-Microsoft NO\n\nNO YES (Up to 10)\n\n“...trained on all languages that appear in public repos- itories" (Fine-tuned)\n\nProgramming Languages work best with (according to the vendor)\n\nMultipurpose (other than programming) Subscription\n\nCan be Used Offline? Can it Access Local Files?\n\nYES\n\nChatGPT Free ChatGPT Plus ($20 per month)\n\nNO NO\n\nC#, Python, and TypeScript\n\nJava,\n\nJavaScript,\n\nNO\n\nFree Preview\n\nNO YES\n\nC, C++, C#, Go, Java, JavaScript, PHP, Python, Ruby, Scala, and Type- Script NO\n\nCopilot for Students (Free) Copilot for Individuals ($10 per month) Copilot for Business ($19 per user, per month) NO YES\n\nTable 1 Comparing Relevant Code Generation Tools\n\n2.2 Amaz

In [10]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_150386/2321294980.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
     #   shutil.rmtree(CHROMA_PATH)


    # Ensure the directory exists and has the correct permissions
    #os.makedirs(CHROMA_PATH, exist_ok=True)
    #os.chmod(CHROMA_PATH, 0o755)
        
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


In [12]:
save_to_chroma(chunks)

Saved 2569 chunks to 2409.15180/chroma.


/tmp/ipykernel_150386/2702161365.py:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


### Use database

In [13]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [14]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

/tmp/ipykernel_150386/244422769.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


### Other parameters
https://medium.com/@callumjmac/implementing-rag-in-langchain-with-chroma-a-step-by-step-guide-16fc21815339

In [15]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
Write your answer in about 2000 words.
"""

In [16]:
def query_rag(query_text,k=3):
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=k)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  llm = Ollama(model = "mistral",temperature=0)
  # Generate response text based on the prompt
  response_text = llm.invoke(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources} \nSourceText:{context_text}"
  return formatted_response, response_text



In [17]:
paper_id

'2409.15180'

In [18]:
def get_topic(file_name):
    # Step 1: Read the JSON file
    with open(file_name + '.json', 'r') as file:
        json_data = json.load(file)
    return json_data
original = get_topic('dataset/'+paper_id+'data')
topic = original['title']
print(topic)
db.similarity_search_with_relevance_scores(f'what are recent developments in {topic}?', k=2)

A Comprehensive Survey with Critical Analysis for Deepfake Speech Detection


[(Document(metadata={'source': '2409.15180/2308.14970v1.Audio_Deepfake_Detection__A_Survey.pdf', 'start_index': 84962}, page_content='14. https://github.com/asvspoof-challenge/2021/tree/main/LA/\n\nBaseline-RawNet2\n\n15. https://github.com/clovaai/aasist\n\n8 FUTURE DIRECTIONS\n\nAlthough some significant progress on audio deepfake de- tection have been made over the last decade, there still exist some limitations which should be addressed in future work. Some potential research directions are summarized as follows.\n\nCollecting audio datasets in the wild: Most of the audio deepfake detection datasets are not collected in the wild, which do not quite match with the real utterances recorded or generated in realistic conditions. The real conditions of the utterances may be even worse and vary more greatly than the simulated conditions. In order to assess audio deepfake detection methods in practical applications, the utterances with a variety of channels or conditions should be collect

In [19]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=10)
# and finally, inspect our final response!
print(response_text)

Unable to find matching results.
 Title: Recent Developments in Deepfake Speech Detection: A Comprehensive Survey with Critical Analysis

Deepfake speech detection has emerged as an active research topic due to the increasing prevalence of deepfake audio in various domains, including entertainment, politics, and cybersecurity. In this survey paper, we provide a comprehensive overview of recent developments in deepfake speech detection, highlighting key differences across various types of deepfakes, competitions, datasets, features, classifications, and evaluation of state-of-the-art approaches.

1. Types of Deepfake Speech
Deepfake speech can be categorized into three main types: text-to-speech (TTS) deepfakes, voice conversion (VC) deepfakes, and speaker embedding (SE) deepfakes [1]. TTS deepfakes generate synthetic speech from text using neural networks. VC deepfakes modify the voice of a source speaker to resemble that of a target speaker. SE deepfakes manipulate the speaker's ident

In [20]:
print(len(response_text.split(' ')))

779


In [21]:
os.makedirs('summaries/'+paper_id,exist_ok=True)
store_summary(formatted_response,'rag10_large_data') #'rag10'

In [22]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=5)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag5_large_data') #rag5

Unable to find matching results.
 Title: Recent Developments in Deepfake Speech Detection: A Comprehensive Survey with Critical Analysis

Deepfake technology, which allows generating synthetic media that can mimic human voices, has gained significant attention in recent years due to its potential applications and malicious uses. Deepfake speech detection has emerged as a critical research area to combat the spread of deepfake audio content, especially in areas such as digital forensics, cybersecurity, and multimedia processing. In this survey, we provide a comprehensive analysis of recent developments in deepfake speech detection, focusing on the methodologies, techniques, and challenges.

Firstly, it is essential to understand the basics of deepfake speech generation. Deepfake speech is created by training generative models such as WaveNet or Tacotron2 on large datasets of real speech recordings. These models learn to generate new speech samples that are similar to the original data b

In [23]:
print(len(response_text.split(' ')))

957


In [24]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=8)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag8_large_data') #rag8

Unable to find matching results.
 Title: Recent Developments in Deepfake Speech Detection: A Comprehensive Survey with Critical Analysis

Deepfake technology, which enables generating synthetic media that can mimic the voice or speech of real individuals, has gained significant attention in recent years due to its potential applications and risks. Deepfake speech detection has emerged as an active research topic to address the challenges posed by this technology. In this survey paper, we aim to provide a comprehensive overview of recent developments in deepfake speech detection, focusing on competitions, datasets, features, classifications, and evaluation.

Deepfake Speech: An Overview
Deepfake speech refers to synthetic audio that mimics the voice or speech of real individuals. Deepfake speech can be generated using various techniques such as text-to-speech (TTS) synthesis, voice conversion, and speaker embedding. These techniques use deep learning models to learn patterns from large 

In [25]:
len(response_text.split(' '))

741

In [34]:
len('Hi what is the concept?'.split(' '))

5